In [ ]:
import urllib
from Bio import Entrez
from xml.dom import minidom
import xml.etree.ElementTree as ET
import pandas as pd
import os
import json

current_folder = os.getcwd()
print (current_folder)

In [ ]:
data_file = str(current_folder) +  "/All_7226_articles_withPMIDs.csv"
data_df = pd.read_csv(data_file)
print (len(data_df))
RCT_filtered = data_df
display(RCT_filtered.head(4))


In [ ]:
RCT_filtered["Abstract_Check"] = ""
RCT_filtered["Title_Check"] = ""

count = 0

for index, row in RCT_filtered.iterrows():
    count +=1
    if count%100 == 0:
        print (count)
    PMID = row["PMID Matched"]
    link = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&api_key=1a7d18b4a3b452b6f1d688fa1ad4282bfa09&retmode=xml&id=" + str(PMID)
    request = urllib.request.Request(link)
    result = urllib.request.urlopen(request)
    tree = ET.parse(result)
    doc = tree.getroot()
    
    #get title
    title = doc.find(".//ArticleTitle")
    if title is not None:
        if title != "":
            article_title = title.text
        else:
            article_title = ""
    else:
        article_title = ""
#     print (article_title)
    
    # get abstract
    abstract = doc.find(".//Abstract")
    article_abstract = ""
    if abstract:
        for abstracttext in abstract.getchildren():
            if abstracttext.tag == "AbstractText":
                abstracttext_children = abstracttext.getchildren()
                if len (abstracttext_children) <=1:
                    if "Label" in abstracttext.attrib:
                        article_abstract += (abstracttext.attrib["Label"]) + ": "
                        article_abstract += str(abstracttext.text) + " "
                    else:
                        article_abstract += str(abstracttext.text) + " "
                else:
                    for children in abstracttext_children:
                        article_abstract += str(children.text) + " "
                        article_abstract += str(children.tail) + " "  
    RCT_filtered.at[index, "Abstract_Check"] = (article_abstract)
    RCT_filtered.at[index, "Title_Check"]= article_title
    
display(RCT_filtered.head(10))

In [ ]:
RCT_filtered.at[index, "PMID Mapping Error"] = ""

for index, row in RCT_filtered.iterrows():
    title_cochrane = row["Title"]
    title_PMC = row["Title_Check"]
    
    # Remove special characters from a string
    title_cochrane = ''.join(filter(str.isalnum, title_cochrane)).lower()
#     print(title_cochrane)
    
    title_PMC = ''.join(filter(str.isalnum, title_PMC)).lower()
#     print(title_PMC)
    
    if title_cochrane in title_PMC:
        RCT_filtered.at[index, "PMID Mapping Error"] = ""
    else:
        RCT_filtered.at[index, "PMID Mapping Error"] = "mapping error"
        

In [ ]:
RCT_filtered.to_csv("Tagger_Results_HighScores_7226_articles_mapping_error_checked.csv")